In [1]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical

In [2]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
from google.colab import drive
drive.mount('/content/gdrive')
path_to_csv = '/content/gdrive/My Drive/Sentiment.csv'
data = pd.read_csv(path_to_csv)

# Select only the necessary columns 'text' and 'sentiment'
mask = data.columns.isin(['text', 'sentiment'])
data = data.loc[:, mask]

Mounted at /content/gdrive


In [3]:
# Keeping only the necessary columns
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [4]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

In [5]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix

In [6]:
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons

In [7]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model

In [8]:
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test

In [9]:
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 65s - loss: 0.8144 - accuracy: 0.6507 - 65s/epoch - 224ms/step
144/144 - 2s - loss: 0.7488 - accuracy: 0.6719 - 2s/epoch - 12ms/step
0.7487729787826538
0.6719091534614563


In [10]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [11]:
model.save('sentimentAnalysis.h5') #Saving the model

In [12]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [13]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [14]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")

1/1 - 0s - 273ms/epoch - 273ms/step
[0.5941629  0.16130976 0.24452734]
Neutral


2. Apply GridSearchCV on the source code provided in the class


In [16]:
from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

<ipython-input-16-6c99b49150f4>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 133s - loss: 0.8280 - accuracy: 0.6468 - 133s/epoch - 179ms/step
186/186 - 2s - loss: 0.7457 - accuracy: 0.6799 - 2s/epoch - 13ms/step


744/744 - 122s - loss: 0.8273 - accuracy: 0.6482 - 122s/epoch - 164ms/step
186/186 - 2s - loss: 0.7717 - accuracy: 0.6783 - 2s/epoch - 13ms/step


744/744 - 118s - loss: 0.8239 - accuracy: 0.6447 - 118s/epoch - 158ms/step
186/186 - 2s - loss: 0.7390 - accuracy: 0.6826 - 2s/epoch - 13ms/step


744/744 - 119s - loss: 0.8257 - accuracy: 0.6463 - 119s/epoch - 159ms/step
186/186 - 2s - loss: 0.7632 - accuracy: 0.6728 - 2s/epoch - 13ms/step


...

Epoch 1/2
186/186 - 37s - loss: 0.8409 - accuracy: 0.6345 - 37s/epoch - 200ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6881 - accuracy: 0.7043 - 26s/epoch - 140ms/step
47/47 - 1s - loss: 0.7369 - accuracy: 0.6735 - 745ms/epoch - 16ms/step


Epoch 1/2
186/186 - 36s - loss: 0.8478 - accuracy: 0.6305 - 36s/epoch - 195ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6837 - accuracy: 0.7077 - 26s/epoch - 138ms/step
47/47 - 1s - loss: 0.7405 - accuracy: 0.6934 - 732ms/epoch - 16ms/step


Epoch 1/2
186/186 - 36s - loss: 0.8480 - accuracy: 0.6317 - 36s/epoch - 195ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6852 - accuracy: 0.7038 - 26s/epoch - 141ms/step
47/47 - 1s - loss: 0.7469 - accuracy: 0.6808 - 754ms/epoch - 16ms/step


Epoch 1/2
186/186 - 39s - loss: 0.8438 - accuracy: 0.6347 - 39s/epoch - 208ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6802 - accuracy: 0.7110 - 26s/epoch - 140ms/step
47/47 - 1s - loss: 0.7828 - accuracy: 0.6755 - 764ms/epoch - 16ms/step


Epoch 1/2
233/233 - 42s - loss: 0.8278 - accuracy: 0.6394 - 42s/epoch - 182ms/step
Epoch 2/2
233/233 - 34s - loss: 0.6794 - accuracy: 0.7141 - 34s/epoch - 146ms/step
Best: 0.681803 using {'batch_size': 40, 'epochs': 2}
